In [ ]:
!scala -version

Scala code runner version 2.12.10 -- Copyright 2002-2019, LAMP/EPFL and Lightbend, Inc.


In [ ]:
!pyspark --packages graphframes:graphframes:0.5.0-spark2.1-s_2.11

Python 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-abf71a7f-5ff1-4baa-bdae-a037c3f41321;1.0
	confs: [default]
	found graphframes#graphframes;0.5.0-spark2.1-s_2.11 in spark-packages
	found com.typesafe.scala-logging#scala-logging-api_2.11;2.1.2 in central
	found com.typesafe.scala-logging#scala-logging-slf4j_2.11;2.1.2 in central
	found org.scala-lang#scala-reflect;2.11.0 in central
	found org.slf4j#slf4j-api;1.7.7 in central
downloading https://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.5.0-spark2.1-s_2.11/graphframes-0.5.0-spark2.1-s_2.11.j

In [ ]:
from graphframes import *

ModuleNotFoundError: No module named 'graphframes'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Jupyter BigQuery Storage')\
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [ ]:
tags_table = 'bigquery-public-data.stackoverflow.tags'
df = spark.read \
  .format("bigquery") \
  .option("table", tags_table) \
  .load()
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- tag_name: string (nullable = true)
 |-- count: long (nullable = true)
 |-- excerpt_post_id: long (nullable = true)
 |-- wiki_post_id: long (nullable = true)



In [ ]:
df_preview = df.select('tag_name', 'count')
df_preview.orderBy('count', ascending=False).show(20)

+-------------+-------+
|     tag_name|  count|
+-------------+-------+
|   javascript|2081682|
|         java|1709763|
|       python|1528649|
|           c#|1433486|
|          php|1369355|
|      android|1296865|
|         html|1019597|
|       jquery| 996292|
|          c++| 690400|
|          css| 682048|
|          ios| 639447|
|        mysql| 606332|
|          sql| 558201|
|            r| 362851|
|      asp.net| 358592|
|      node.js| 348330|
|       arrays| 340725|
|            c| 338564|
|ruby-on-rails| 320468|
|         .net| 300816|
+-------------+-------+
only showing top 20 rows



In [ ]:
df_preview.orderBy('count', ascending=True).show(20)

+---------------+-----+
|       tag_name|count|
+---------------+-----+
|           esql|    1|
|       topcased|    1|
|           pugs|    1|
|           bduf|    1|
|          crowd|    1|
|  curl-language|    1|
|    design-debt|    1|
|           jscc|    1|
|      firefox11|    1|
|         bpgsql|    1|
|           kmem|    1|
|       korundum|    1|
|    startup.cmd|    1|
|       llvm-3.1|    1|
|     slimpicker|    1|
|       tracekit|    1|
|     cg-toolkit|    1|
| mapi-audiocopy|    1|
|mapi-audiopaste|    1|
|            ryz|    1|
+---------------+-----+
only showing top 20 rows



In [ ]:
post_table = 'bigquery-public-data.stackoverflow.stackoverflow_posts'
df = spark.read \
  .format("bigquery") \
  .option("table", post_table) \
  .load()
df.printSchema()

root
 |-- id: long (nullable = false)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- accepted_answer_id: long (nullable = true)
 |-- answer_count: long (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- community_owned_date: timestamp (nullable = true)
 |-- creation_date: timestamp (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- last_activity_date: timestamp (nullable = true)
 |-- last_edit_date: timestamp (nullable = true)
 |-- last_editor_display_name: string (nullable = true)
 |-- last_editor_user_id: long (nullable = true)
 |-- owner_display_name: string (nullable = true)
 |-- owner_user_id: long (nullable = true)
 |-- parent_id: long (nullable = true)
 |-- post_type_id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- tags: string (nullable = true)
 |-- view_count: long (nullable = true)



In [ ]:
df.count()

31017889

In [ ]:
from pyspark.sql.functions import col, length
df_prev_pqs = df.select('id', 'tags')\
    .where((col('tags').isNotNull()) & (length(col('tags')) > 0))
df_prev_pqs.show(10)

+--------+--------------------+
|      id|                tags|
+--------+--------------------+
|  525063|android|url|andro...|
|14128723|        java|eclipse|
|   20728|winforms|deployme...|
| 6531409|          ios4|uikit|
|  593205| c#|delegates|action|
|  377901|java|web-services...|
|  638886|java|html|applet|...|
|  235982|javascript|syntax...|
|  687884|    xsd|guid|xsd.exe|
|  373234|            perforce|
+--------+--------------------+
only showing top 10 rows



In [ ]:
df_prev_pqs.count()

11846518

In [ ]:
df_prev_pqs.select('*') \
    .groupBy('tags') \
    .count().orderBy(col('count'), ascending=False) \
    .show(10)

+-----------------+-----+
|             tags|count|
+-----------------+-----+
|          android|89107|
|             java|64807|
|              php|64504|
|javascript|jquery|64155|
|       javascript|61253|
|           jquery|49771|
|           python|46085|
|         html|css|45411|
|               c#|43906|
|        php|mysql|40867|
+-----------------+-----+
only showing top 10 rows



In [ ]:
#df_prev_pqs['tag_lst'] = df_prev_pqs\
#    .apply(lambda row: row['tags'].split('|'))
#df_prev_pqs.show(10)
import pyspark


split_col = pyspark.sql.functions.split(df_prev_pqs['tags'], "\\|")
df_prev_pqs = df_prev_pqs.withColumn('tag_lst', split_col)
df_prev_pqs.show(10)

+--------+--------------------+--------------------+
|      id|                tags|             tag_lst|
+--------+--------------------+--------------------+
|  525063|android|url|andro...|[android, url, an...|
|14128723|        java|eclipse|     [java, eclipse]|
|   20728|winforms|deployme...|[winforms, deploy...|
| 6531409|          ios4|uikit|       [ios4, uikit]|
|  593205| c#|delegates|action|[c#, delegates, a...|
|  377901|java|web-services...|[java, web-servic...|
|  638886|java|html|applet|...|[java, html, appl...|
|  235982|javascript|syntax...|[javascript, synt...|
|  687884|    xsd|guid|xsd.exe|[xsd, guid, xsd.exe]|
|  373234|            perforce|          [perforce]|
+--------+--------------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import count, explode

df_prev_pqs.select('*', explode("tag_lst").alias("exploded"))\
    .where(col("exploded").isin(["java", "html"]))\
    .groupBy("tag_lst").count().show(10)

# df_prev_pqs.select("*", explode("tag_lst").alias("exploded"))\
#     .where(col("exploded") == "java" & col("exploded") == "html")\
#     .show(10)

+--------------------+-----+
|             tag_lst|count|
+--------------------+-----+
|         [php, html]| 8446|
|[java, random, im...|    1|
|[html, windows-ph...|    1|
|[java, android, url]|   46|
|[java, jsp, array...|    1|
|[javascript, html...|   22|
|[java, android, f...|    3|
|[php, mysql, html...|   12|
|[html, css, mater...|    5|
|[java, swing, bor...|   28|
+--------------------+-----+
only showing top 10 rows



In [ ]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="tag_lst", minSupport=0.00001, minConfidence=0.6)
model = fpGrowth.fit(df_prev_pqs)

In [ ]:
# Display frequent itemsets.
model.freqItemsets.orderBy('freq', ascending=False).show(10)

+------------+-------+
|       items|   freq|
+------------+-------+
|[javascript]|1143864|
|      [java]|1086506|
|        [c#]| 962755|
|       [php]| 933606|
|   [android]| 848769|
|    [jquery]| 744879|
|    [python]| 587679|
|      [html]| 545002|
|       [c++]| 452908|
|       [ios]| 438169|
+------------+-------+
only showing top 10 rows



In [ ]:
# Display generated association rules.
model.associationRules.show(10)

+--------------------+---------------+------------------+------------------+
|          antecedent|     consequent|        confidence|              lift|
+--------------------+---------------+------------------+------------------+
|[fancybox, javasc...|       [jquery]| 0.799470549305096|12.714739243303553|
| [dgrid, javascript]|         [dojo]|0.9477611940298507|1309.8075180560102|
|               [awt]|         [java]|0.9802494802494802| 10.68796961292999|
|[django-template-...|       [django]|0.9577464788732394| 93.75045157869538|
| [mschart, winforms]|           [c#]|0.8502994011976048|10.462773147557423|
|    [routes, devise]|[ruby-on-rails]|0.8177966101694916| 39.40727560042899|
|[apache, regex, php]|  [mod-rewrite]|0.7716981132075472| 358.0299047810467|
|[apache, regex, php]|    [.htaccess]|0.8584905660377359| 202.1170146547205|
|        [enums, wpf]|           [c#]|0.6732673267326733| 8.284426988123142|
|    [viewmodel, wpf]|         [mvvm]|              0.68|455.01763669227296|

In [ ]:
model.associationRules\
    .orderBy('confidence', ascending=False)\
    .show(10)

+--------------------+-----------+------------------+------------------+
|          antecedent| consequent|        confidence|              lift|
+--------------------+-----------+------------------+------------------+
|            [wix3.7]|      [wix]|               1.0| 4400.949063564131|
|[angularjs-contro...|[angularjs]|               1.0|173.93826509726514|
|           [lattice]|        [r]|               1.0| 220.9660549674441|
|       [awt, applet]|     [java]|               1.0|28.548290575477722|
|        [data.table]|        [r]|               1.0| 220.9660549674441|
|         [sitecore7]| [sitecore]|               1.0| 6476.903111296721|
|                [s4]|        [r]|               1.0| 220.9660549674441|
|[iron-router, jav...|   [meteor]|               1.0|1431.7050080775443|
|         [sitecore6]| [sitecore]|0.9990654205607477| 6470.849930818874|
|[ember-data, java...| [ember.js]|0.9989339019189766|1629.4936044207047|
+--------------------+-----------+-----------------

In [ ]:
model.associationRules \
    .where(col('confidence') > 0.8) \
    .orderBy('confidence', ascending=True) \
    .show(10)

+--------------------+--------------------+------------------+------------------+
|          antecedent|          consequent|        confidence|              lift|
+--------------------+--------------------+------------------+------------------+
|        [filewriter]|              [java]|0.8002853067047075|  22.8467774790913|
|  [mod-rewrite, php]|         [.htaccess]|0.8003224939532384|  493.348588609339|
|[task-parallel-li...|                [c#]| 0.800332778702163|25.784995448317854|
|[devise, ruby-on-...|     [ruby-on-rails]|0.8004201680672269|100.98820360257152|
| [textbox, winforms]|                [c#]|0.8005272407732865|25.791260596706405|
|[build.gradle, an...|           [android]|0.8005738880918221|29.256619877882624|
|[jquery-ui-tabs, ...|            [jquery]|0.8008130081300813|  33.3470657595864|
|[expandablelistad...|[expandablelistview]|  0.80083857442348| 9826.076743823474|
|[extjs4.1, javasc...|             [extjs]|0.8010204081632653|1213.2409837952466|
|              [

In [ ]:
from pyspark.sql.functions import size
model.associationRules \
    .where((size(col('antecedent')) == 1) & (col('confidence') > 0.8)) \
    .orderBy('confidence', ascending=True) \
    .show(10)

+--------------------+--------------------+------------------+------------------+
|          antecedent|          consequent|        confidence|              lift|
+--------------------+--------------------+------------------+------------------+
|        [filewriter]|              [java]|0.8002853067047075|  22.8467774790913|
|[task-parallel-li...|                [c#]| 0.800332778702163|25.784995448317854|
|[expandablelistad...|[expandablelistview]|  0.80083857442348| 9826.076743823474|
|              [qtip]|            [jquery]|0.8010204081632653|33.355702210886406|
|             [flex3]|              [flex]|0.8011343495214462|1017.3379320209376|
|             [scapy]|            [python]|0.8011834319526627| 42.28672244702762|
|[atlassian-source...|               [git]|0.8012820512820513|346.81882874508426|
|   [powershell-v2.0]|        [powershell]|0.8017771701982228| 710.2509001268766|
|[uicollectionview...|               [ios]|0.8019851116625311| 56.77235307199047|
|              [

In [ ]:
model.associationRules \
    .where((size(col('antecedent')) == 1)) \
    .count()

2200

In [ ]:
model.freqItemsets \
    .where(size(col('items')) == 1) \
    .count()

7136

In [ ]:
for row in model.associationRules.iterrows():
    print(row)
    break

AttributeError: 'DataFrame' object has no attribute 'iterrows'

In [ ]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df_prev_pqs).show()

+--------+--------------------+--------------------+----------+
|      id|                tags|             tag_lst|prediction|
+--------+--------------------+--------------------+----------+
|  525063|android|url|andro...|[android, url, an...|        []|
|14128723|        java|eclipse|     [java, eclipse]|        []|
|   20728|winforms|deployme...|[winforms, deploy...|      [c#]|
| 6531409|          ios4|uikit|       [ios4, uikit]|        []|
|  593205| c#|delegates|action|[c#, delegates, a...|        []|
|  377901|java|web-services...|[java, web-servic...|        []|
|  638886|java|html|applet|...|[java, html, appl...|     [css]|
|  235982|javascript|syntax...|[javascript, synt...|        []|
|  687884|    xsd|guid|xsd.exe|[xsd, guid, xsd.exe]|     [xml]|
|  373234|            perforce|          [perforce]|        []|
|   11318|.net|winforms|ani...|[.net, winforms, ...|      [c#]|
|  520030|    c#|linq|distinct|[c#, linq, distinct]|        []|
|15301999|   python|python-2.7|[python, 

In [ ]:
question_table = 'bigquery-public-data.stackoverflow.posts_questions'
df = spark.read \
  .format("bigquery") \
  .option("table", question_table) \
  .load()
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- accepted_answer_id: long (nullable = true)
 |-- answer_count: long (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- community_owned_date: timestamp (nullable = true)
 |-- creation_date: timestamp (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- last_activity_date: timestamp (nullable = true)
 |-- last_edit_date: timestamp (nullable = true)
 |-- last_editor_display_name: string (nullable = true)
 |-- last_editor_user_id: long (nullable = true)
 |-- owner_display_name: string (nullable = true)
 |-- owner_user_id: long (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- post_type_id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- tags: string (nullable = true)
 |-- view_count: long (nullable = true)



In [ ]:
df.count()

20094655